# 3: Dataset creation for Phrase Grounding

In [1]:
import os
import pickle
import sqlite3
import shutil

import cv2
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

Extract the Photo table from the database

In [2]:
BCU_database_folder = '../data/BCU_database/'
dataset_for_phrase_grounding_folder = '../data/dataset_for_phrase_grounding/'

In [10]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect(BCU_database_folder + 'bcu_caption.db')
Photo = pd.read_sql_query("SELECT * from Photo", con)

# Verify that result of SQL query is stored in the dataframe
print(Photo.keys())
print('original number of picture :',len(Photo))

Index(['id', 'bcu_id', 'title', 'title_en', 'caption', 'year_from', 'year_to',
       'photographer_name', 'fund_name', 'locality', 'theme', 'filename',
       'title_en_preprocessed', 'caption_preprocessed'],
      dtype='object')
original number of picture : 2216


In [11]:
Photo = Photo.loc[Photo.filename.notnull() & (Photo.caption_preprocessed.notna() & Photo.title_en_preprocessed.notna())]
len(Photo)

1500

In [12]:
dataset = []
for id, row in Photo.iterrows():
    elem = dict()
    elem['image_id'] = row['id']
    elem['caption'] = dict()
    try :
        elem['caption']['raw'] = row['caption']
    except:
        elem['caption']['raw'] = ''
    try :
        elem['caption']['preprocessed'] = row['caption_preprocessed'].lower()
    except:
        elem['caption']['preprocessed'] = ''
    elem['title'] = dict()
    try :
        elem['title']['raw'] = row['title_en']
    except:
        elem['title']['raw'] = ''
    try :
        elem['title']['preprocessed'] = row['title_en_preprocessed'].lower()
    except:
        elem['title']['preprocessed'] = ''

    elem['filename'] = row['filename']
    dataset.append(elem)

    # img = cv2.imread('./data/03_resized/' + row['filename'])
    # plt.imshow(cv2.bitwise_not(img))
    # plt.show()

In [18]:
pickle.dump(dataset,open(dataset_for_phrase_grounding_folder + 'dataset.p','wb'))

for elem in dataset:
    shutil.copyfile(BCU_database_folder + '03_resized/' + elem['filename'], dataset_for_phrase_grounding_folder + 'img/' +  elem['filename'])
shutil.make_archive('../data/dataset_for_phrase_grounding', 'zip', dataset_for_phrase_grounding_folder)

'/home/tgieruc/Heritage-in-the-digital-age/data/dataset_for_phrase_grounding.zip'